Задача: необходимо разработать модель, которая будет "сравнивать" названия компаний из двух столбцов таблицы и выдавать являются ли они "одинаковыми". Реализовать поисковой движок по данному датасету. Тестовых данных не будет, нужно разделить полученный датасет на трейн, вал, и тест.

---

Содержание:

0. Загрузка и обзор данных
1. Очистка данных
2. Бейзлайн - TF-IDF + LogReg
3. Решение членов команды

# 0. Загрузка и обзор данных

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, classification_report, roc_auc_score

from sklearn.linear_model import LogisticRegression

In [ ]:
#!gdown 1hr1XGnhXskK7GXTw5bw8_-kkWHBcBMf_ # for colab

In [2]:
#df = pd.read_csv('/content/train.csv') # for colab
df = pd.read_csv('train.csv')

In [3]:
df.shape

(497819, 4)

In [4]:
df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [5]:
df['is_duplicate'].mean() * 100

0.7348052203712594

Датасет совсем не сбалансированный, только 0,7% положительного класса

In [6]:
df['pair_id'].isna().sum()

0

In [7]:
df['is_duplicate'].isna().sum()

0

Пропусков нет, отлично. Посмотрим на примеры положительного класса

In [8]:
df_one = df[df['is_duplicate'] == 1]
df_one.sample(15)

,pair_id,name_1,name_2,is_duplicate
138766,138767,Basf Corp.,Basf De Costa Rica Sociedad Anonima,1
43394,43395,Shenyang Bridgestone,Bridgestone Canada Inc.,1
454132,454133,SOPREMA Polska Sp. z o.o.,SOPREMA DRUMMONDVILLE,1
353107,353108,"JOWAT SE""/ЧЕРЕЗ ТЕРМИНАЛ В ПОЛЬШЕ","ООО""ЙОВАТ""",1
35161,35162,"Arlanxeo Singapore Pte., Ltd. Tearlanxeo Canad...",Arlanxeo Canada Inc.,1
96861,96862,"Bridgestone India Pvt., Ltd.",Bridgestone Tire,1
141401,141402,"Michelin (Shenyang) Tyre Co., Ltd.",Michelin Tyre Plc,1
475546,475547,Ceva Freight Llc,Ceva Freight Management,1
247344,247345,Sumitomo Corp. Of America,Sumitomo Corporation Do Brasil Sa,1
75466,75467,Pirelli Pneus Ltda,Pirelli Neumaticos Sa De Cv Boulev,1


Как видно, названия компаний:
1. Написаны на разных языках - по хорошему надо привести все к одному языку - англ.
2. Присутствует указание страны - необходимо удалить названия стран
3. Сокращенные названия компаний Ltd., Co., и т.д. - необходимо удалить их
4. Привести все к одному регистру, убрать знаки препинания, спец символы () \ и т.д.
5. Удалить цифры и адреса - это вопрос надо подробней разобрать

---

# 1 Очистка данных

## 1.1 Перевод

Попробуем несколько методов посмотрим какой быстрее работает

### 1.1.1 Translatepy - 321 stars on Github

In [ ]:
!pip install translatepy

In [ ]:
from translatepy.translators.google import GoogleTranslate
translator = GoogleTranslate()

Проверка работы

In [ ]:
df['name_1'][24518]

In [ ]:
a = translator.language(df['name_1'][24518])
print(str(a.result))

In [ ]:
gtranslate = GoogleTranslate()
test = gtranslate.translate(df['name_1'][24518], 'eng')
print(str(test.result))

Отлично, теперь напишем функцию для применения к всему датасету

In [ ]:
def translate (word):
  '''
  Translate words into English if they are in another language.
  '''
  try:
    temp = translator.language(word)
    if str(temp.result) != 'eng':
      tr = gtranslate.translate(word, 'eng').result
    else:
      tr = word
  except Exception:
    tr = word
    
  return str(tr)

Засечем время выполнения

In [ ]:
%%time
df['name_1_tr'] = df['name_1'].apply(lambda x: translate(x))
# терпения хватило только на 3 часа, не успело закончить и выключил

In [ ]:
%%time
df['name_2_tr'] = df['name_2'].apply(lambda x: translate(x))
# эту ячейку даже не запускал

In [ ]:
temp_df = df.loc[:999,:]

In [ ]:
%%time
temp_df['name_1_tr'] = temp_df['name_1'].apply(lambda x: translate(x))
# посмотрим за сколько получится перевести 1000 строк

Итого на 500к строк понадобится в хкудшем случае 25 часов 8(

### 1.1.2 Deep-translator - 709 stars on Github

В данной библиотеке также можно использовать различные переводчики от именнитых фирм (для некоторых типа Яндекса требуется api ключ). Для определения языка нужно иметь api-key от Detect Language, то есть используется сторонний сервис для определения языка. А без определения языка переводить 500к строк займет еще больше времени. Проверим это.

In [ ]:
!pip install -U deep-translator

In [ ]:
from deep_translator import GoogleTranslator

In [ ]:
translated = GoogleTranslator(source='auto', target='en').translate(df['name_1'][24518])

In [ ]:
translated

In [ ]:
%%time
temp_df['name_1_tr'] = temp_df['name_1'].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))
# посмотрим за сколько получится перевести 1000 строк

In [ ]:
temp_df

Данная библиотека работает быстрее, весь датасет она переведет за 16,6 часов. Быстрее, чем прошлая библиотека, но для наших целей все равно медленно =(

### 1.1.3 No Language Left Behind(NLLB) - 19,4k stars on Github

Новая сеть выпущенная компанией Meta AI. No Language Left Behind (NLLB) — это первый в своем роде прорывной проект в области искусственного интеллекта, который открывает модели с открытым исходным кодом, способные обеспечить высококачественный перевод напрямую между любой парой из более чем 200 языков, включая малоресурсные языки, такие как астурийский и лугандский, урду и многие другие.

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

In [ ]:
translator = pipeline('translation', model=model, tokenizer=tokenizer, max_length=1000, src_lang="rus_Cyrl", tgt_lang='eng_Latn')

In [ ]:
translator(df['name_1'][24518])

В этой модели нужно обязательно указывать исходный язык, сама определять егго модель не умеет =( поэтому нужно городить костыль с определением языка и дальше подавать язык источника для перевода в данной модели. Что на наш взгляд будет занимать большое количество времени по сравнению с ранее рассмотреными библиотеками

---

Надо признать, что идея с переводом не совсем удачная, так как перевод занимает большое количество времени. Кусочек датасета брать не логично, так как необходимо будет реализовать поисковой движок по всему датасету.

## 1.2 Удаление названия стран (Максим)

In [9]:
!pip install pycountry

In [10]:
import csv
import pycountry

In [11]:
continents_names = ['Asia', 'Africa', 'North America', 'South America', 'Antarctica', 'Europe', 'Australia']

continents_names_regex = '|'.join([continent for continent in continents_names])

countries_names_regex = '|'.join([country.name for country in pycountry.countries])
countries_names_regex = re.sub(' ', '|', countries_names_regex)
countries_names_regex = re.sub("[^A-Za-z-|]", "", countries_names_regex)

full_regex = continents_names_regex + countries_names_regex

pattern = re.compile(full_regex, re.UNICODE | re.IGNORECASE)

# file_name = '/content/train.csv'
file_name = 'train.csv'
labels_dict = dict(pair_id=[], name_1=[], name_2=[], is_duplicate=[])

In [12]:
with open(file_name, encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        result = [pattern.sub("", company).strip() for company in row]
        labels_dict['pair_id'].append(result[0])
        labels_dict['name_1'].append(result[1])
        labels_dict['name_2'].append(result[2])
        labels_dict['is_duplicate'].append(result[3])

df = pd.DataFrame(labels_dict)

In [13]:
df.sample(15)

,pair_id,name_1,name_2,is_duplicate
347591,347592,Ryder Capital S De Rl De Cv,A S International,0
268580,268581,Bridgestone Inc.,Bridgestone Firestone De Sa De Cv,1
375040,375041,Pon Pure Chemical,Akzo Nob Chemicals Sa De Cv,0
36188,36189,Han's Sports,Sportech Ltd.,0
419213,419214,Pak Petrochemical Indtries (Pvt),"Shenhua Chemical Indtrial Co., Ltd.",0
258842,258843,Bliz Indtria Comercio Importacao & Exportacao Eir,Salient Indtries Ltd.,0
266289,266290,Afi International Inc.,Jas Jet Air Service S.A.,0
327162,327163,Km Trading Co Llc,"Hee Trading Co., Ltd. Ningbo Hao",0
151933,151934,Ktm Ag,"Kumar Adhesive Pvt., Ltd.",0
306261,306262,Scanwl Logistics (Chi) Inc.,Csll Logistics,0


## 1.3 Удаление спец. символов и сокращений (Денис)

In [14]:
#сокращения компаний
companies = ['ltd', 'ltda', 'saic', 's', 'a', 'i', 'c', 'co', 'ag', 'r', 'o', 
             'p', 'group', 'inc', 'sp', 'ооо', 'зао', 'ncr', 'cv', 'limited',
             'sa', 'spa', 'pte', 'pvt', 'gmbh', 'nv', 'imp', 'corp', 'pt', 
             'mfg', 'do', 'l', 't', 'd', 'corporation', 'corp', 'doo', 'do', 
             'bv', 'de', 'llc', 'sti', 'c', 'inds', 'industriesusa', 'holdings',
             'sas', 'ad', 'kg', 'srl', 'sociedad', 'anoni', 'private', 'bhd', 
             'alo', 'asrc', 'lc', 'holding', 'rl', 'ca', 'na', 'llp', 'b', 'on',
             'tic', 've', 'san', 'sl', 'roppongi', 'zoo', 'lt', 'z', 'oo', 'gmb',
             'h', 'apm', 'cd', 'as', 'pty', 'kft', 'fm', 'sdn', 'vat', 'id', 'nl',
             'the', 'tse', 'tax', 'be', 'lp', 'cokg', 'v', 'филиал', 'компании',
             'компани', 'ccp', 'research', 'and', 'development', 'center', 'tld',
             'f', 'u', 'lda', 'plc', 'm', 'общество', 'с', 'ограниченной',
             'ответственностью', 'ieo', 'cie', 'k', 'e', 'ind', 'рус', 'ао',
             'лтд', 'cs', 'industrial', 'se', 'через', 'терминал', 'в']

In [15]:
# обработка датасета
def clean_func(s):
  s = s.lower()
  s = re.sub(r'[\W\d]', ' ', s) #r'[^a-z]'
  s = s.split()
  for i in range(len(s)):
    if s[i] in companies:
      s[i] = ''
    else:
      continue
  s = ' '.join(s).strip()
  return s

In [16]:
final_df1 = df.name_1.apply(clean_func).to_frame()
final_df2 = df.name_2.apply(clean_func).to_frame()

In [17]:
final_df = pd.concat([final_df1, final_df2, df.is_duplicate], axis=1)

In [18]:
final_df.sample(15)

,name_1,name_2,is_duplicate
121344,eastern exp,ayesha exp,0
66323,albemarle,coim,0
467685,brascola em recuperacao judicial,cp international,0
31515,ammbs indtries,disrupt indtries,0
136744,xiamen shenglin plastic,siemens,0
162152,farrow global logistics,allsop,0
52712,anwer rashid indtries,rbn indtries,0
6163,dongguan xu ke ectronic products,dongguan jinggan plastic products,0
299249,wearbeeta,eet,0
471022,speed,polymers,0


# 2 Baseline

В качестве безйлайна выбрали TF-IDF & LogReg, в качестве метрики будем ориентироваться на ROC-AUC.
Но сначала разделим наш датасет на 3 выборки: train, val, test в соотношении 80,10,10

In [19]:
final_df['full_name'] = final_df['name_1'] + ' ' + final_df['name_2']

In [21]:
df_train, df_temp = train_test_split(final_df, test_size=0.2, stratify=final_df['is_duplicate'], random_state=101)
df_valid, df_test = train_test_split(df_temp, test_size=0.5, stratify=df_temp['is_duplicate'], random_state=101)

## 2.1 TF-IDF

In [22]:
count_tf_idf = TfidfVectorizer(ngram_range=(1,5), analyzer='char_wb', max_features=10000)

Данные для обучения

In [23]:
features_train = df_train.drop(['is_duplicate'], axis=1)
corpus = df_train['full_name'].values.astype('U')
features_train = count_tf_idf.fit_transform(corpus)
target_train = df_train['is_duplicate']

Данные для валидации

In [24]:
features_valid = df_valid.drop(['is_duplicate'], axis=1)
corpus = df_valid['full_name'].values.astype('U')
features_valid = count_tf_idf.transform(corpus)
target_valid = df_valid['is_duplicate']

Данные для тестирования

In [25]:
features_test = df_test.drop(['is_duplicate'], axis=1)
corpus = df_test['full_name'].values.astype('U')
features_test = count_tf_idf.transform(corpus)
target_test = df_test['is_duplicate']

## 2.2 LogisticRegression

In [26]:
%%time
logreg = LogisticRegression(solver='sag', max_iter=100, random_state=101)

tuned_parameters = {"C": np.logspace(-2, 2, 10)}

best_logreg = GridSearchCV(logreg, param_grid=tuned_parameters, scoring='roc_auc',
                           cv=5, n_jobs=-1).fit(features_train, target_train)

target_pred = best_logreg.predict_proba(features_valid)[:, 1]
score_val = roc_auc_score(target_valid, target_pred)

print("Detailed classification report:") # Дополнительно вывыдем данный отчет
target_pred_1 = best_logreg.predict(features_valid)
print(classification_report(target_valid, target_pred_1))

Detailed classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     49416
           1       0.79      0.56      0.66       366

    accuracy                           1.00     49782
   macro avg       0.89      0.78      0.83     49782
weighted avg       1.00      1.00      1.00     49782

CPU times: total: 16.3 s
Wall time: 2min 30s


### 2.3 ROC-AUC

Создадим таблицу, куда будем заносить результаты. Сначала результат baseline, а далее результаты членов команды

In [27]:
result_df = pd.DataFrame(columns=['ML_name', 'ROC_AUC valid', 'ROC_AUC test'])

In [28]:
target_pred2 = best_logreg.predict_proba(features_test)[:, 1]
score_test = roc_auc_score(target_test, target_pred2)

In [29]:
result_df.loc[0]=['TF-IDF + LogReg', round(score_val, 6), round(score_test, 6)]
result_df.head()

,ML_name,ROC_AUC valid,ROC_AUC test
0,TF-IDF + LogReg,0.987668,0.988127


# 3 Решения членов команды

Мы решили попробовать несколько решений для данной задачи

## 3.1 Денис

## 3.2 Кирилл

In [30]:
!pip install catboost
!pip install optuna

In [31]:
from catboost import CatBoostClassifier
import optuna

from sklearn.feature_extraction.text import CountVectorizer

In [32]:
final_df_2 = final_df.copy()

In [33]:
final_df_2['full_name'] = final_df_2['name_1'] + ' ' + final_df_2['name_2']

In [34]:
df_train_2, df_temp_2 = train_test_split(final_df_2, test_size=0.2, random_state=101)
df_valid_2, df_test_2 = train_test_split(df_temp_2, test_size=0.5, random_state=101)

In [35]:
cv = CountVectorizer(ngram_range=(1, 5), max_features=10000)

Данные для обучения

In [36]:
features_train_2 = df_train_2.drop(['is_duplicate'], axis=1)
corpus_2 = df_train_2['full_name'].values.astype('U')
features_train_2 = cv.fit_transform(corpus_2)
target_train_2 = df_train_2['is_duplicate']

Данные для валидации

In [37]:
features_valid_2 = df_valid_2.drop(['is_duplicate'], axis=1)
corpus_2 = df_valid_2['full_name'].values.astype('U')
features_valid_2 = cv.transform(corpus_2)
target_valid_2 = df_valid_2['is_duplicate']

Данные для тестирования

In [38]:
features_test_2 = df_test_2.drop(['is_duplicate'], axis=1)
corpus_2 = df_test_2['full_name'].values.astype('U')
features_test_2 = cv.transform(corpus_2)
target_test_2 = df_test_2['is_duplicate']

In [39]:
def objective(trial):

  param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 4, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"]),
        "used_ram_limit": "32gb",
        "eval_metric": "AUC",
        "random_state": 101,
        "logging_level": "Silent"
    }

  if param["bootstrap_type"] == "Bayesian":
    param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
  elif param["bootstrap_type"] == "Bernoulli":
    param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)
    
  best_cat = CatBoostClassifier(**param)

  best_cat.fit(features_train_2, target_train_2, 
               eval_set=(features_valid_2, target_valid_2), 
               early_stopping_rounds=20)
  target_pred_2 = best_cat.predict_proba(features_valid_2)[:, 1]
  score_val = roc_auc_score(target_valid_2, target_pred_2)

  return score_val

Поиск гиперпараметров занимает огромное количество времени больше 4 часов. Поэтому не будем его запускать, а воспользуемся уже найдеными параметрами.

In [40]:
# %%time
# study = optuna.create_study(study_name=f'catboost-seed{101}', direction="maximize")
# study.optimize(objective, n_trials=100, n_jobs=-1)

Лучший скор:

In [41]:
# study.best_value

Лучшие параметры:

In [42]:
# study.best_params

In [43]:
# Запомним полученные параметры Best is trial 31 with value: 0.9719550556418812.
# {'objective': 'Logloss', 
#  'colsample_bylevel': 0.054188649478630675, 
#  'depth': 10, 
#  'boosting_type': 'Ordered', 
#  'bootstrap_type': 'Bernoulli', 
#  'subsample': 0.2330707011865564}

Создадим заново параметры и обучим модель, чтобы сделать предсказания для теста. Использовать сразу лучшую модель из **optuna** для предсказаний (типа как в GridSearch), можно, но очень заморочено. Проще заново обучить модель с найдеными параметрами и сделать предсказания.

In [44]:
# Задаем параметры, добавляя неоходимые параметры
params = {'objective': 'Logloss',
          'colsample_bylevel': 0.054188649478630675,
          'depth': 10, 
          'boosting_type': 'Ordered', 
          'bootstrap_type': 'Bernoulli', 
          'subsample': 0.2330707011865564}

params['eval_metric'] = 'AUC'
params['random_state'] = 101
params['logging_level'] = 'Silent'

In [48]:
# Обучаем модель с лучшими параметрами
best_cat = CatBoostClassifier(**params).fit(features_train_2, target_train_2, 
                                            eval_set=(features_valid_2, target_valid_2), 
                                            early_stopping_rounds=20)
target_pred_2 = best_cat.predict_proba(features_valid_2)[:, 1]
score_val_2 = roc_auc_score(target_valid_2, target_pred_2)
print(score_val_2)

print("Detailed classification report:") # Дополнительно вывыдем данный отчет
target_pred_2 = best_cat.predict(features_valid_2)
print(classification_report(target_valid_2, target_pred_2))

0.9719550556418812
Detailed classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     49427
           1       0.98      0.82      0.89       355

    accuracy                           1.00     49782
   macro avg       0.99      0.91      0.95     49782
weighted avg       1.00      1.00      1.00     49782



Заносим очередной результат

In [49]:
target_pred_2_1 = best_cat.predict_proba(features_test_2)[:, 1]
score_test_2 = roc_auc_score(target_test_2, target_pred_2_1)

In [50]:
result_df.loc[2]=[' CountVectorizer + Catboost', round(score_val_2, 6), round(score_test_2, 6)]
result_df.head()

,ML_name,ROC_AUC valid,ROC_AUC test
0,TF-IDF + LogReg,0.987668,0.988127
2,CountVectorizer + Catboost,0.971955,0.969178


## 3.3 Максим

Скопируем данные для обучения, валидации и теста в отдельные переменные для вариант fasttext + logreg

In [53]:
final_df_ft = final_df.copy()
df_train_ft = df_train.copy()
df_temp_ft = df_temp.copy()
df_valid_ft = df_valid.copy()
df_test_ft = df_test.copy()

In [54]:
!pip install gensim==4.2.0

  Attempting uninstall: gensim
    Found existing installation: gensim 4.1.2
    Uninstalling gensim-4.1.2:
      Successfully uninstalled gensim-4.1.2


In [55]:
import nltk
from nltk.tokenize import word_tokenize
import gensim
from gensim.models import FastText

In [56]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Дима\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

Функция для разбивки предложения на отдельные слова

In [57]:
def tokenized_sentence(sentence):
    words = []
    if not sentence: 
      words.append(' ')
    else:
      for word in word_tokenize(sentence):
        words.append(word) 
    return words

Обучаем fasttext

In [58]:
corpus = df_train_ft['full_name'].apply(lambda x: tokenized_sentence(x))
fast_text_model = gensim.models.FastText(corpus, min_count = 1, window = 5)

Векторное представление данных

In [59]:
def sent_vector(word):
    if not word: word = [' ']
    return np.mean(fast_text_model.wv[word], axis=0)

Данные для обучения

In [60]:
features_train = [sent_vector(word) for word in corpus]
target_train = df_train_ft['is_duplicate']

Данные для валидации

In [61]:
corpus = df_valid_ft['full_name'].apply(lambda x: tokenized_sentence(x))
features_valid = [sent_vector(word) for word in corpus]
target_valid = df_valid_ft['is_duplicate']

Данные для теста

In [62]:
corpus = df_test_ft['full_name'].apply(lambda x: tokenized_sentence(x))
features_test = [sent_vector(word) for word in corpus]
target_test = df_test_ft['is_duplicate']

Логистическая регрессия

In [65]:
logreg_ft = LogisticRegression(solver='sag', max_iter=1000, random_state=101)

tuned_parameters = {"C": np.logspace(-2, 2, 10)}

best_logreg = GridSearchCV(logreg_ft, param_grid=tuned_parameters, scoring='roc_auc',
                           cv=5, n_jobs=-1).fit(features_train, target_train)

target_pred_valid = best_logreg.predict_proba(features_valid)[:, 1]
score_val_ft = roc_auc_score(target_valid, target_pred_valid)

target_pred_test = best_logreg.predict_proba(features_test)[:, 1]
score_test_ft = roc_auc_score(target_test, target_pred_test)

Добавляем результат в таблицу

In [66]:
result_df.loc[3]=['FastText + LogReg', round(score_val_ft, 6), round(score_test_ft, 6)]
result_df.head()

,ML_name,ROC_AUC valid,ROC_AUC test
0,TF-IDF + LogReg,0.987668,0.988127
2,CountVectorizer + Catboost,0.971955,0.969178
3,FastText + LogReg,0.949958,0.937622


# 4 Поиск соседнего вектора

Поиск дубликата будем искать через евклидово расстояние между Вводимым значением и колонкой `name_1`

In [106]:
temp_df = final_df.sample(30)
temp_df.head()

,name_1,name_2,is_duplicate,full_name
202074,khawaja establishment,seabiz international,0,khawaja establishment seabiz international
235920,ld global logistics,allsop,0,ld global logistics allsop
70450,super tex indtries,eaton indtries,0,super tex indtries eaton indtries
320861,eurovia czechy,repsol,0,eurovia czechy repsol
466716,stecore agement services,gee sales,0,stecore agement services gee sales


In [107]:
search_name = 'overseas transporte y'

In [108]:
search_tab = temp_df['name_1']

In [109]:
corpus = search_tab.values.astype('U')
search_tab_v = count_tf_idf.transform(corpus)
search_name_v = count_tf_idf.transform(np.array(list(search_name)))

In [110]:
from sklearn.metrics import pairwise_distances

In [111]:
distances = [pairwise_distances(search_tab_v[x], search_name_v) for x in range(search_tab_v.shape[0])]

best_index = np.array(distances).argsort()[1]

print("Индекс наиболее совпадающей компании:", best_index)

Индекс наиболее совпадающей компании: [[ 8 19  4  7 13  6 11 17  9  0 15 18  2  5 16 10  3 14 12 20  1]]


# Итоговые результаты и вывод

Показать табличку и написать словами, какой результат лучший